In [ ]:
%pip install -qU python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from supabase.client import Client, create_client
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import SupabaseVectorStore

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_store = SupabaseVectorStore(
        client=supabase, embedding=embedding, table_name="documents", query_name="match_documents_embeddings")


The following function need to be created in Supabase's public schema:

In [ ]:
DROP FUNCTION match_documents_embeddings;
CREATE FUNCTION match_documents_embeddings(query_embedding vector(768),
                                            match_count int)
    RETURNS TABLE(
        id bigint,
        content text,
        metadata jsonb,
        embedding vector(768),
        similarity float)
    LANGUAGE plpgsql
    AS $$
    # variable_conflict use_column
BEGIN
    RETURN query
    SELECT
        id,
        content,
        metadata,
        embedding,
        1 -(documents.embedding <=> query_embedding) AS similarity
    FROM
        documents
    ORDER BY
        documents.embedding <=> query_embedding
    LIMIT match_count;
END;
$$;

In [ ]:
def print_docs(docs):
    for k, doc in enumerate(docs):
        print(f"\n{k+1}) {doc.page_content}")

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr")
docs = retriever.get_relevant_documents("What is Flutter?")
print_docs(docs)